# Ex - GroupBy

### Introduction:

GroupBy can be summarized as Split-Apply-Combine.

Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

Check out this [Diagram](http://i.imgur.com/yjNkiwL.png)  
### Step 1. Import the necessary libraries

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,317 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,687 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,230 kB]
Get:13 http://archive.ubuntu.com/u

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv).

### Step 3. Assign it to a variable called drinks.

In [2]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv', sep = ',')
drinks = spark.createDataFrame(df)

drinks.show(5)

+-----------+-------------+---------------+-------------+----------------------------+---------+
|    country|beer_servings|spirit_servings|wine_servings|total_litres_of_pure_alcohol|continent|
+-----------+-------------+---------------+-------------+----------------------------+---------+
|Afghanistan|            0|              0|            0|                         0.0|       AS|
|    Albania|           89|            132|           54|                         4.9|       EU|
|    Algeria|           25|              0|           14|                         0.7|       AF|
|    Andorra|          245|            138|          312|                        12.4|       EU|
|     Angola|          217|             57|           45|                         5.9|       AF|
+-----------+-------------+---------------+-------------+----------------------------+---------+
only showing top 5 rows



### Step 4. Which continent drinks more beer on average?

In [5]:
drinks.groupBy('continent').avg('beer_servings').orderBy('avg(beer_servings)').show()

+---------+------------------+
|continent|avg(beer_servings)|
+---------+------------------+
|       AS| 37.04545454545455|
|       AF|61.471698113207545|
|       OC|           89.6875|
|      NaN|145.43478260869566|
|       SA|175.08333333333334|
|       EU|193.77777777777777|
+---------+------------------+



### Step 5. For each continent print the statistics for wine consumption.

In [10]:
from pyspark.sql.functions import sum, avg, min, max, stddev

drinks.groupBy('continent').agg(sum('wine_servings'),avg('wine_servings'),min('wine_servings'),max('wine_servings'),stddev('wine_servings')).show()

+---------+------------------+------------------+------------------+------------------+---------------------+
|continent|sum(wine_servings)|avg(wine_servings)|min(wine_servings)|max(wine_servings)|stddev(wine_servings)|
+---------+------------------+------------------+------------------+------------------+---------------------+
|       SA|               749|62.416666666666664|                 1|               221|    88.62018888937148|
|      NaN|               564| 24.52173913043478|                 1|               100|   28.266378301658847|
|       AS|               399| 9.068181818181818|                 0|               123|   21.667033931944488|
|       OC|               570|            35.625|                 0|               212|    64.55578982554547|
|       EU|              6400|142.22222222222223|                 0|               370|    97.42173756146494|
|       AF|               862|16.264150943396228|                 0|               233|    38.84641897335842|
+---------

### Step 6. Print the mean alcohol consumption per continent for every column

In [12]:
drinks.groupBy('continent').avg().show()

+---------+------------------+--------------------+------------------+---------------------------------+
|continent|avg(beer_servings)|avg(spirit_servings)|avg(wine_servings)|avg(total_litres_of_pure_alcohol)|
+---------+------------------+--------------------+------------------+---------------------------------+
|       SA|175.08333333333334|              114.75|62.416666666666664|                6.308333333333334|
|      NaN|145.43478260869566|   165.7391304347826| 24.52173913043478|                5.995652173913045|
|       AS| 37.04545454545455|   60.84090909090909| 9.068181818181818|               2.1704545454545454|
|       OC|           89.6875|             58.4375|            35.625|               3.3812499999999996|
|       EU|193.77777777777777|  132.55555555555554|142.22222222222223|                8.617777777777778|
|       AF|61.471698113207545|  16.339622641509433|16.264150943396228|               3.0075471698113208|
+---------+------------------+--------------------+----

### Step 7. Print the median alcohol consumption per continent for every column

In [34]:
#still figuring out logic for SQL query

### Step 8. Print the mean, min and max values for spirit consumption.
#### This time output a DataFrame

In [32]:
mean = drinks.groupBy('continent').avg('spirit_servings').show()
min = drinks.groupBy('continent').min('spirit_servings').show()
max = drinks.groupBy('continent').max('spirit_servings').show()

+---------+--------------------+
|continent|avg(spirit_servings)|
+---------+--------------------+
|       SA|              114.75|
|      NaN|   165.7391304347826|
|       AS|   60.84090909090909|
|       OC|             58.4375|
|       EU|  132.55555555555554|
|       AF|  16.339622641509433|
+---------+--------------------+

+---------+--------------------+
|continent|min(spirit_servings)|
+---------+--------------------+
|       SA|                  25|
|      NaN|                  68|
|       AS|                   0|
|       OC|                   0|
|       EU|                   0|
|       AF|                   0|
+---------+--------------------+

+---------+--------------------+
|continent|max(spirit_servings)|
+---------+--------------------+
|       SA|                 302|
|      NaN|                 438|
|       AS|                 326|
|       OC|                 254|
|       EU|                 373|
|       AF|                 152|
+---------+--------------------+

